# FractalN Mushroom Classification

This notebook runs the training process in Google Colab using the main.py script.

## Setup
First, let's clone the repository and install dependencies:

In [ ]:
# First check CUDA version
!nvidia-smi
!nvcc --version

# Clone repository
!git clone https://github.com/sapienskid/FractalN.git
%cd FractalN

# Add src directory to Python path
import sys
sys.path.append('./src')



# Install correct CuPy version based on CUDA
import subprocess
cuda_output = subprocess.getoutput('nvcc --version')
if 'release 11' in cuda_output:
    !pip install cupy-cuda11x
elif 'release 12' in cuda_output:
    !pip install cupy-cuda12x
else:
    !pip install cupy-cuda110

# Install base requirements first
!pip install -r requirements.txt

# Verify setup
import cupy as cp
print(f"\nSetup verification:")
print(f"CuPy version: {cp.__version__}")
print(f"CUDA available: {cp.cuda.is_available()}")
print("\nRepository contents:")
!ls
print("\nDataset contents:")
!ls data/processed_mushroom_dataset

## GPU Check
Let's verify GPU availability:

In [ ]:
!nvidia-smi

In [ ]:
# Verify CuPy can access GPU
import cupy as cp

# Test GPU computation
x = cp.array([1, 2, 3])
print(f"CuPy GPU available: {cp.cuda.is_available()}")
if cp.cuda.is_available():
    device = cp.cuda.Device(0)
    print(f"GPU Device: {device.attributes['DEVICE_NAME'].decode('utf-8')}")
    print(f"Total Memory: {device.mem_info[1]/1024**3:.2f} GB")

# Test basic computation
y = cp.square(x)
print(f"\nTest computation on GPU: {x} squared = {y}")
print("GPU test successful!")

## GPU Performance Test
Let's run a computation test to verify GPU acceleration:

In [ ]:
import cupy as cp
import numpy as np
import time

def test_gpu_computation():
    # Create large arrays for testing
    size = 10000000
    
    # CPU computation using NumPy
    print("Starting CPU computation...")
    start_cpu = time.time()
    x_cpu = np.random.random(size)
    y_cpu = np.random.random(size)
    z_cpu = x_cpu + y_cpu
    cpu_time = time.time() - start_cpu
    
    # GPU computation using CuPy
    print("\nStarting GPU computation...")
    start_gpu = time.time()
    x_gpu = cp.random.random(size)
    y_gpu = cp.random.random(size)
    z_gpu = x_gpu + y_gpu
    cp.cuda.Device().synchronize()  # Wait for GPU to finish
    gpu_time = time.time() - start_gpu
    
    # Print results with color formatting
    print("\n" + "="*50)
    print("GPU Performance Test Results:")
    print("="*50)
    print(f"CPU time: {cpu_time:.4f} seconds")
    print(f"GPU time: {gpu_time:.4f} seconds")
    print(f"GPU speedup: {cpu_time/gpu_time:.2f}x")
    
    # Print GPU information
    print("\nGPU Information:")
    print("-"*30)
    print(f"Current Device: {cp.cuda.Device().id}")
    print(f"Device Name: {cp.cuda.runtime.getDeviceProperties(cp.cuda.Device().id)['name'].decode()}")
    print(f"Memory Usage: {cp.cuda.runtime.memGetInfo()[0]/1024**3:.2f} GB free of {cp.cuda.runtime.memGetInfo()[1]/1024**3:.2f} GB total")

# Run the test
try:
    test_gpu_computation()
except Exception as e:
    print(f"\nError: {str(e)}")
    print("\nTroubleshooting tips:")
    print("1. Make sure you're using a GPU runtime in Colab (Runtime > Change runtime type > GPU)")
    print("2. Verify CuPy installation was successful")
    print("3. Check if CUDA version matches your GPU")

## Training
Now we'll run the training using main.py:

In [ ]:
# Run training using properly imported modules
from src.gpu_utils import configure_gpu
from src.main import train

# Configure GPU with memory pool
configure_gpu()

# Run training
train()